In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNet, NeuralNetRegressor
from sklearn.metrics import mean_squared_error as mse, mean_absolute_percentage_error as mape
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
import gc
import matplotlib.pyplot as plt
from tqdm.auto import tqdm, trange
from sklearn.base import BaseEstimator, TransformerMixin
from skorch.callbacks import EarlyStopping, LRScheduler
from skorch.dataset import Dataset
from skorch.helper import predefined_split
import joblib
from skorch.callbacks import Callback
import time
#from helpful_functions import InputLogTransformer, OutputLogTransformer, build_neural_network, make_datasets
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def log_transform(X): # transforms first column of data.  Alternativly, this could be implemented via operations on DataFrames
    X1 = X.copy()
    X1[:, 0] = np.log(X1[:, 0])
    return X1
def log_inverse(X):
    X1 = X.copy()
    X1[:, 0] = np.exp(X1[:,0])
    return X1

class OutputLogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._estimator = StandardScaler()
    def fit(self, y):
        y_copy = y.copy()
        y_copy = np.log(y_copy)
        self._estimator.fit(y_copy)
        
        return self
    def transform(self, y):
        y_copy = y.copy()
        y_copy = np.log(y_copy)
        return self._estimator.transform(y_copy)
    def inverse_transform(self, y):
        y_copy = y.copy()
        y_reverse = np.exp(self._estimator.inverse_transform(y_copy))
        
        return y_reverse

class InputLogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._estimator = StandardScaler()
    def fit(self, X):
        X_copy = X.copy()
        X_copy = log_transform(X_copy)
        self._estimator.fit(X_copy)
        
        return self
    def transform(self, X):
        X_copy = X.copy()
        X_copy = log_transform(X_copy)
        return self._estimator.transform(X_copy)
    def inverse_transform(self, X):
        X_copy = X.copy()
        X_reverse = log_inverse(self._estimator.inverse_transform(X_copy))
        
        return X_reverse
    
class LDIAModel(nn.Module):
    '''
    Perceptron Model of variable architecture for hyperparameter tuning
    '''
    def __init__(self, n_hidden = 1,n_neurons=64,activation=nn.LeakyReLU()):
        super().__init__()
        self.norms = []
        self.layers = []
        self.acts = []
        self.norm0 = nn.BatchNorm1d(3)
        self.layer0 = nn.Linear(3,n_neurons)
        for i in range(1,n_hidden+1):
            self.norms.append(nn.BatchNorm1d(n_neurons))
            self.acts.append(activation)
            self.add_module(f"norm{i}", self.norms[-1])
            self.add_module(f"act{i}", self.acts[-1])
            if (i != n_hidden):
                self.layers.append(nn.Linear(n_neurons, n_neurons))
                self.add_module(f"layer{i}", self.layers[-1])
        self.output = nn.Linear(n_neurons, 3)

    def forward(self, x):
        '''
          Forward pass
        '''
        x = self.layer0(self.norm0(x))
        for norm, layer, act in zip(self.norms, self.layers, self.acts):
            x = act(layer(norm(x)))
        return self.output(x)
    
def build_neural_network(max_epochs=100, n_hidden=3, n_neurons=32, activation=nn.LeakyReLU(), device=torch.device('cuda'),loss_fn=nn.MSELoss(), optimizer=optim.Adam, lr=1e-2, shuffled=True, batch_size=1024, patience=5, gamma=0.95,valid_ds=None,compiled=False):
    return NeuralNetRegressor(
    module=LDIAModel,
    max_epochs = max_epochs,
    module__n_hidden=n_hidden,
    module__n_neurons = n_neurons,
    module__activation=activation,
    device=device,
    criterion = loss_fn,
    optimizer=optimizer,
    optimizer__lr = lr,
    iterator_train__shuffle=shuffled,
    batch_size=batch_size,
    callbacks=[('early_stopping', EarlyStopping(patience=patience,monitor='valid_loss')),
    ('lr_scheduler', LRScheduler(policy='ExponentialLR',gamma=gamma))],
    train_split=predefined_split(valid_ds),
    compile = compiled 
)

def make_datasets(X, y, *, train_size=0.8, random_state=42):
    X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=train_size,random_state=random_state)
    input_transformer = InputLogTransformer()
    output_transformer = OutputLogTransformer()
    X_train = input_transformer.fit_transform(X_train)
    X_val = input_transformer.transform(X_val)
    y_train = output_transformer.fit_transform(y_train)
    y_val = output_transformer.transform(y_val)
    #train_ds = Dataset(X_train, y_train)
    #valid_ds = Dataset(X_val, y_val)
    return X_train, y_train, X_val, y_val, input_transformer, output_transformer

In [2]:
noise = 30 # ADJUST level of gaussian noise added to outputs
mod_type = 'nn'
description = mod_type + '_noise-' + str(noise)
filename = '../datasets/fuchs_v3-2_seed-5_points_25000_noise_' + str(noise) + '.csv'  # CHANGE TO DESIRED DATA FILE
df = pd.read_csv(filename)

input_list = ['Intensity_(W_cm2)', 'Target_Thickness (um)', 'Focal_Distance_(um)'] # independent variables
output_list = ['Max_Proton_Energy_(MeV)', 'Total_Proton_Energy_(MeV)', 'Avg_Proton_Energy_(MeV)',
               'Max_Proton_Energy_Exact_(MeV)', 'Total_Proton_Energy_Exact_(MeV)', 'Avg_Proton_Energy_Exact_(MeV)'] # training outputs
X = np.array(df[input_list].copy(), dtype=np.float32)
y = np.array(df[output_list].copy(), dtype=np.float32)
    
train_split = 0.8 # Reserve 80% of entire dataset for training
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_split, shuffle = False)
y_train = y_train[:, 0:3]

pct = 50 # e.g. 10% is 2,000 points training
len_df = int(len(X_train)*(pct/100))
X_train = X_train[0:len_df]
y_train = y_train[0:len_df]

X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled, input_transformer, output_transformer = make_datasets(X_train, y_train, random_state=42)
train_ds = Dataset(X_train_scaled, y_train_scaled)
valid_ds = Dataset(X_val_scaled, y_val_scaled)

In [12]:
num_inputs = 3
num_outputs = 3
n_hidden = 3
n_neurons = 64
train_split = 0.8 # Reserve 80% of entire dataset for training
batch_size = 256
γ = 0.95
max_epochs = 100
lr=1e-3

nn_model = build_neural_network(max_epochs=max_epochs, 
                                 n_hidden=n_hidden, 
                                 n_neurons=n_neurons, 
                                 activation=nn.LeakyReLU(), 
                                 device=device,loss_fn=nn.MSELoss(), 
                                 optimizer=optim.Adam, 
                                 lr=lr, 
                                 shuffled=True, 
                                 batch_size=batch_size, 
                                 patience=5, 
                                 gamma=γ, 
                                 valid_ds=valid_ds)
    
nn_model.fit(train_ds, y=None)

y_train_predict = nn_model.predict(X_train_scaled)
y_train_predict_unscaled = output_transformer.inverse_transform(y_train_predict)

# Corrections due to Log Scaling
y_train = output_transformer.inverse_transform(y_train_scaled)
correction_factor = np.mean(y_train/y_train_predict_unscaled, axis=0) 
y_train_predict_corrected = y_train_predict_unscaled*correction_factor
print(correction_factor, np.sqrt(1+(noise/100)**2))
for j in range(num_outputs):
    print(mape(y_train[:, j], y_train_predict_corrected[:, j]))

  epoch    train_loss    valid_loss      lr     dur
-------  ------------  ------------  ------  ------
      1        0.3381        0.0654  0.0010  0.1988
      2        0.0478        0.0412  0.0009  0.1280
      3        0.0428        0.0413  0.0009  0.1281
      4        0.0430        0.0396  0.0009  0.1262
      5        0.0414        0.0394  0.0008  0.1245
      6        0.0442        0.0387  0.0008  0.1242
      7        0.0437        0.0380  0.0007  0.1252
      8        0.0415        0.0404  0.0007  0.1241
      9        0.0422        0.0386  0.0007  0.1277
     10        0.0414        0.0386  0.0006  0.1244
     11        0.0412        0.0383  0.0006  0.1258
Stopping since valid_loss has not improved in the last 5 epochs.
[1.0384319 1.0345069 1.032342 ] 1.044030650891055
0.25720128
0.26341668
0.25660914


In [4]:
def model(X):
    X_scaled = input_transformer.transform(X.copy().reshape(-1, 3))
    energies = output_transformer.inverse_transform(nn_model.predict(X_scaled))
    E_max = energies[:, 0]
    E_tot = energies[:, 1]
    E_avg = energies[:, 2]

    return E_max, E_tot, E_avg

def generate_random_points(bounds, n):
    np.random.seed(0)
    points = []
    for bound in bounds:
        points.append(np.random.uniform(bound[0], bound[1], n))
    return np.array(points, dtype=np.float32).transpose()

In [5]:
bounds = [(1e19, 1e19), (0.5, 10.0), (0, 10.0)]
n_points = 100000
points = generate_random_points(bounds, n_points)
Emax, Etot, Eavg = model(points)
output_df = pd.DataFrame(columns=['Intensity', 'Thickness', 'Offset', 'E Max', 'E Tot', 'E Avg'])
output_df['Intensity'] = points[:, 0]
output_df['Thickness'] = points[:, 1]
output_df['Offset'] = points[:, 2]
output_df['E Max'] = Emax
output_df['E Tot'] = Etot
output_df['E Avg'] = Eavg
output_df.head()

,Intensity,Thickness,Offset,E Max,E Tot,E Avg
0,1.000000e+19,5.584942,2.177699,0.326697,84157568.0,0.081434
1,1.000000e+19,9.088420,5.872497,0.125710,28633386.0,0.034329
2,1.000000e+19,5.272768,9.876133,0.174977,38419624.0,0.045561
3,1.000000e+19,1.458265,9.508671,0.521847,124839792.0,0.109516
4,1.000000e+19,5.512029,3.429715,0.312105,79200744.0,0.078045


In [6]:
output_df.to_csv('predictions_dfs/{}_noise={}_train_pts={}.csv'.format(mod_type, noise, len_df), index=False)